In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import data
from dataset import rdDataset
from model import rdcnn
from math import log10


# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
# cudnn.benchmark = True
path = './data1'

# Parameters

params = {'test_split': .25,
          'shuffle_dataset': True,
          'batchsize': 2,
          'testBatchsize': 2,
          'random_seed': 42,
          'numworkers':16}
max_epoches = 200
learning_rate = 1e-3
drop_rate = 0.0

print('===> Loading datasets')
# Load All Dataset
dataset = rdDataset(path)

# Creating data indices for training and validation splits:
training_data_loader, testing_data_loader = data.DatasetSplit(dataset, **params)

print('===> Building model')
model = rdcnn(drop_rate).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5)



def train(epoch):
    epoch_loss = 0
    for iteration, batch in enumerate(training_data_loader, 1):
        input, target = batch[0].to(device, torch.float), batch[1].to(device, torch.float)
        optimizer.zero_grad()
        loss = criterion(model(input), target)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

#         print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(training_data_loader), loss.item()))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(training_data_loader)))
    return epoch, epoch_loss / len(training_data_loader)
    
def test():
    avg_error = 0
    avg_loss = 0
    with torch.no_grad():
        for batch in testing_data_loader:
            input, target = batch[0].to(device, torch.float), batch[1].to(device, torch.float)

            prediction = model(input)
            tmp_error = 0
#             print(len(prediction))
            for j in range(len(prediction)):
                tmp_error += torch.mean((prediction[j]-target[j])**2/torch.max(target[j]))
            avg_error += tmp_error / len(prediction)
            mse = criterion(prediction, target)
            avg_loss += mse
    print("===> Avg. Loss: {:.4f} ".format(avg_loss / len(testing_data_loader)))
    print("===> Avg. Error: {:.4f} ".format(avg_error / len(testing_data_loader)))
    return avg_loss / len(testing_data_loader),avg_error / len(testing_data_loader)

def checkpoint(epoch):
    model_out_path = "./checkpoint_largedata2/model_epoch_{}.pth".format(epoch)
    torch.save(model, model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))
    
    

===> Loading datasets
===> Building model


In [4]:
L_train_loss = []
L_test_loss = []
L_test_error = []
for epoch in range(1, max_epoches + 1):
    train_loss = train(epoch)
    test_loss,test_error = test()
    checkpoint(epoch)
#     data.TestErrorPlot(model,device, testing_data_loader)
    L_train_loss.append(train_loss)
    L_test_loss.append(test_loss)
    L_test_error.append(test_error)

===> Epoch 1 Complete: Avg. Loss: 0.0587
===> Avg. Loss: 0.0578 
===> Avg. Error: 0.0633 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_1.pth
===> Epoch 2 Complete: Avg. Loss: 0.0558
===> Avg. Loss: 0.0576 
===> Avg. Error: 0.0628 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_2.pth
===> Epoch 3 Complete: Avg. Loss: 0.0553
===> Avg. Loss: 0.0570 
===> Avg. Error: 0.0646 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_3.pth
===> Epoch 4 Complete: Avg. Loss: 0.0553
===> Avg. Loss: 0.0577 
===> Avg. Error: 0.0628 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_4.pth
===> Epoch 5 Complete: Avg. Loss: 0.0550
===> Avg. Loss: 0.0577 
===> Avg. Error: 0.0623 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_5.pth
===> Epoch 6 Complete: Avg. Loss: 0.0550
===> Avg. Loss: 0.0570 
===> Avg. Error: 0.0648 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_6.pth
===> Epoch 7 Complete: Avg. Loss: 0.0550
===> Avg. Loss: 0.0579 
===> Avg. Error: 0.0623

===> Epoch 55 Complete: Avg. Loss: 0.0547
===> Avg. Loss: 0.0568 
===> Avg. Error: 0.0642 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_55.pth
===> Epoch 56 Complete: Avg. Loss: 0.0546
===> Avg. Loss: 0.0568 
===> Avg. Error: 0.0628 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_56.pth
===> Epoch 57 Complete: Avg. Loss: 0.0547
===> Avg. Loss: 0.0568 
===> Avg. Error: 0.0630 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_57.pth
===> Epoch 58 Complete: Avg. Loss: 0.0549
===> Avg. Loss: 0.0568 
===> Avg. Error: 0.0629 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_58.pth
===> Epoch 59 Complete: Avg. Loss: 0.0547
===> Avg. Loss: 0.0567 
===> Avg. Error: 0.0633 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_59.pth
===> Epoch 60 Complete: Avg. Loss: 0.0546
===> Avg. Loss: 0.0572 
===> Avg. Error: 0.0658 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_60.pth
===> Epoch 61 Complete: Avg. Loss: 0.0547
===> Avg. Loss: 0.0571 
===> Avg. 

===> Avg. Loss: 0.0594 
===> Avg. Error: 0.0629 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_108.pth
===> Epoch 109 Complete: Avg. Loss: 0.0546
===> Avg. Loss: 0.0568 
===> Avg. Error: 0.0627 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_109.pth
===> Epoch 110 Complete: Avg. Loss: 0.0546
===> Avg. Loss: 0.0567 
===> Avg. Error: 0.0627 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_110.pth
===> Epoch 111 Complete: Avg. Loss: 0.0546
===> Avg. Loss: 0.0567 
===> Avg. Error: 0.0635 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_111.pth
===> Epoch 112 Complete: Avg. Loss: 0.0546
===> Avg. Loss: 0.0567 
===> Avg. Error: 0.0631 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_112.pth
===> Epoch 113 Complete: Avg. Loss: 0.0546
===> Avg. Loss: 0.0569 
===> Avg. Error: 0.0624 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_113.pth
===> Epoch 114 Complete: Avg. Loss: 0.0547
===> Avg. Loss: 0.0567 
===> Avg. Error: 0.0639 
Checkpoint save

===> Epoch 161 Complete: Avg. Loss: 0.0546
===> Avg. Loss: 0.0567 
===> Avg. Error: 0.0628 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_161.pth
===> Epoch 162 Complete: Avg. Loss: 0.0545
===> Avg. Loss: 0.0569 
===> Avg. Error: 0.0630 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_162.pth
===> Epoch 163 Complete: Avg. Loss: 0.0547
===> Avg. Loss: 0.0567 
===> Avg. Error: 0.0640 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_163.pth
===> Epoch 164 Complete: Avg. Loss: 0.0546
===> Avg. Loss: 0.0568 
===> Avg. Error: 0.0636 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_164.pth
===> Epoch 165 Complete: Avg. Loss: 0.0546
===> Avg. Loss: 0.0570 
===> Avg. Error: 0.0650 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_165.pth
===> Epoch 166 Complete: Avg. Loss: 0.0546
===> Avg. Loss: 0.0567 
===> Avg. Error: 0.0629 
Checkpoint saved to ./checkpoint_largedata2/model_epoch_166.pth
===> Epoch 167 Complete: Avg. Loss: 0.0546
===> Avg. Loss: 0.056

In [ ]:
import importlib 
importlib.reload(data)
data.TestErrorPlot(model,device, testing_data_loader)

In [ ]:
from matplotlib import pyplot as plt
with torch.no_grad():
    for batch in testing_data_loader:
        input, target = batch[0].to(device, torch.float), batch[1].to(device, torch.float)

        prediction = model(input)
        
for t in range(len(prediction)):
    fig, ax = plt.subplots(1,2, figsize=(10,5))

    im = ax[0].imshow(prediction[t][0].cpu(),cmap = "jet")
    im = ax[1].imshow(target[t][0].cpu(),cmap = "jet")

    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.84, 0.27, 0.01, 0.47])
    fig.colorbar(im, cax=cbar_ax)

plt.show()

In [ ]:
plt.figure(figsize=(10,8))

plt.plot(num_epochs_list, error_list)
    
# plt.title("Loss ")
plt.xlabel('Epoch', fontsize=30)
plt.ylabel('accuracy', fontsize=30)

plt.rcParams.update({'font.size': 25})
plt.show()